# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""6275097744559574010""","""../../../../data/SampleHDS.jso…",9.2848e18,"""10306486805967617818""","""2382519158753068455""","""17760177122755181677""",false
"""9482796887042462106""","""../../../../data/SampleHDS.jso…",2.0454e18,"""11918527913091301470""","""2382519158753068455""","""15118512626600884629""",true
"""16061077705285200777""","""../../../../data/SampleHDS.jso…",1.5406e19,"""684820393843911019""","""2981876698801444673""","""17760177122755181677""",false
"""9482796887042462106""","""../../../../data/SampleHDS.jso…",2.3701e18,"""8977965327411525791""","""7636242769622112659""","""17760177122755181677""",false
"""6275097744559574010""","""../../../../data/SampleHDS.jso…",1.4807e19,"""11956642666021257144""","""7636242769622112659""","""13375232735198986275""",true
"""16061077705285200777""","""../../../../data/SampleHDS.jso…",1.4127e19,"""2234553176610934197""","""1101636941703303503""",null,false
"""16061077705285200777""","""../../../../data/SampleHDS.jso…",1.1740e19,"""7232745456979798795""","""7636242769622112659""","""13375232735198986275""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""5766404158110507986""","""../../../../data/SampleHDS.jso…",1.5242e19,"""8504739752384003293""","""FirstMortgage30yr""","""12812892158211725560""","""Rejected"""
"""13367359089129038591""","""../../../../data/SampleHDS.jso…",1.9311e18,"""7638137855293712250""","""FirstMortgage30yr""","""1276008524838559547""","""Accepted"""
"""16430965575142587899""","""../../../../data/SampleHDS.jso…",1.8246e19,"""14462901201731778082""","""MoneyMarketSavingsAccount""","""12812892158211725560""","""Rejected"""
"""13367359089129038591""","""../../../../data/SampleHDS.jso…",9.4900e17,"""17995255738908433474""","""BasicChecking""","""12812892158211725560""","""Rejected"""
"""5766404158110507986""","""../../../../data/SampleHDS.jso…",6.5217e18,"""2897280204769898120""","""BasicChecking""","""70882184920580256""","""Accepted"""
"""16430965575142587899""","""../../../../data/SampleHDS.jso…",1.7656e19,"""7277685732885758086""","""UPlusFinPersonal""",null,"""Rejected"""
"""16430965575142587899""","""../../../../data/SampleHDS.jso…",9.6935e18,"""2080474136168805276""","""BasicChecking""","""70882184920580256""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""3473946129008980297""","""../../../../data/SampleHDS.jso…",1.7191e19,"""12875483311381377467""","""FirstMortgage30yr""","""5534842213215035278""","""Rejected"""
"""3277094176435325323""","""../../../../data/SampleHDS.jso…",1.1160e19,"""15120354705354382329""","""FirstMortgage30yr""","""7986167464058135351""","""Accepted"""
"""17586505008797777738""","""../../../../data/SampleHDS.jso…",1.7163e19,"""6854933321251688621""","""MoneyMarketSavingsAccount""","""5534842213215035278""","""Rejected"""
"""3277094176435325323""","""../../../../data/SampleHDS.jso…",1.9619e18,"""16033506363561669748""","""BasicChecking""","""5534842213215035278""","""Rejected"""
"""3473946129008980297""","""../../../../data/SampleHDS.jso…",1.1535e19,"""13829135170550592963""","""BasicChecking""","""4970695758426312814""","""Accepted"""
"""17586505008797777738""","""../../../../data/SampleHDS.jso…",1.1176e19,"""7186761782723303341""","""UPlusFinPersonal""",null,"""Rejected"""
"""17586505008797777738""","""../../../../data/SampleHDS.jso…",1.4264e18,"""7727667316676853198""","""BasicChecking""","""4970695758426312814""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
filename=filename
Customer_CLV=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""17174759180291830288""",1.2076e19,"""5613992885271300235""","""FirstMortgage30yr""","""11051565088612097061""","""Rejected"""
"""6569120804418583131""",1.0270e19,"""8257885365892022783""","""FirstMortgage30yr""","""13564162851899026182""","""Accepted"""
"""12718880256019266485""",1.8160e19,"""10161459987275879238""","""MoneyMarketSavingsAccount""","""11051565088612097061""","""Rejected"""
"""6569120804418583131""",1.1990e19,"""5396933171168263340""","""BasicChecking""","""11051565088612097061""","""Rejected"""
"""17174759180291830288""",1.6543e19,"""3498391044375404885""","""BasicChecking""","""7488771017295566855""","""Accepted"""
"""12718880256019266485""",1.7700e19,"""6804590939661930777""","""UPlusFinPersonal""",null,"""Rejected"""
"""12718880256019266485""",2.5216e18,"""3249274183386193199""","""BasicChecking""","""7488771017295566855""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'